# Exercise 3
# Navigate the robot between any two points of its environment

--------------------------------------

--------------------------------------
### Initialization. Get data from solved Excercise 1

*Explain how to go over from Excercise 2 to Excercise 3*

--------------------------------------------

--------------------------------------------

### Content:
- <font size="4">**[A. Utils installation and downloads](#A)**</font></br>
>- <font size="3">[A.1. Install SOM and SARSA utils](#A1)</font></br>
>- <font size="3">[A.2. Download robot positions and SOM lattice from your Collab Storage](#A2)</font></br>
- <font size="4">**[B. Perform SOM training (editable) ](#B)**</font></br>
- <font size="4">**[C. Complete the SARSA implementation](#C)**</font></br>
>- <font size="3">[C.1. SARSA description](#C1)</font></br>
>- <font size="3">[C.2. SARSA function (TO DO)](#C2)</font></br>
>- <font size="3">[C.3. Upload SARSA function to your Collab storage](#C3)</font></br>
- <font size="4">**[D. SARSA simulation ](#D)**</font></br>
>- <font size="3">[D.1. Perform SARSA training (editable) ](#D1)</font></br>
>- <font size="3">[D.2. Save SOM training result ](#D2)</font></br>
- <font size="4">**[E. Result evaluation](#E)**</font></br>
- <font size="4">**[F. Robot navigation within NRP platform (SOM & SARSA application)](#F)**</font></br>
>- <font size="3">[F.1. Generate test for NRP platform ](#F1)</font></br>
>- <font size="3">[F.2. Save NRP test ](#F2)</font></br>
>- <font size="3">[F.3. Run experiment within NRP platform](#F3)</font></br>
- <font size="4">**[G. Submission and grading ](#G)**</font></br>

---------------------------------------------------

---------------------------------------------------
## A. Utils installation and download of data from Collab storage<a id='A'></a>

### A.1. Install the SARSA and SOM utils <a id='A1'></a>

In [ ]:
from IPython.display import clear_output
! pip uninstall epflx_robox_nrp_utils -y
! pip install --user --force-reinstall git+https://github.com/HBPNeurorobotics/EPFLx-RoboX-Neurorobotics-utils#egg=epflx_robox_nrp_utils
! pip install --upgrade "hbp-service-client==1.1.1"
clear_output()

### A.2. Download robot positions and SOM lattice from your Collab Storage <a id='A2'></a>
Run the following cell to download the robot positions and the SOM lattice (csv files) from your Collab Storage.

In [ ]:
clients = get_hbp_service_client()
collab_path = get_collab_storage_path()
from os import path

filename = raw_input("(Robot positions) Enter the name of the data file you want to use for simulation:")
if not filename: 
    filename = 'robot_positions.csv'; clear_output()
filepath = path.join(collab_path, filename)
clients.storage.download_file(filepath, './robot_positions.csv')

filename = raw_input("(SOM lattice) Enter the name of the data file you want to use for simulation:")
if not filename: 
    filename = 'SOM_data_lattice.csv'; clear_output()
filepath = path.join(collab_path, filename)
clients.storage.download_file(filepath, './SOM_data_lattice.csv')

In [ ]:
# Utility function used to save files to your Collab storage

"""
Save a file to the Collab storage

:param filepath: relative path to the file
:param filetype: type of the file, e.g., 'text/csv' or 'text/x-python'
:param remove: remove the file of the Jupyter notebook space if True 
"""
def save_to_collab_storage(filename, filetype, remove=False):
    from os import path
    clients = get_hbp_service_client()
    collab_path = get_collab_storage_path()
    filename = path.basename(filepath)
    # if you create new file
    if os.path.isfile(filename):
        # if old version of the file exists
        filepath = os.path.join(collab_name, filename)
        if clients.storage.exists(filepath): 
            clients.storage.delete(filepath)
        pydata = clients.storage.upload_file(filename, filepath, filetype)
        # remove 
        if remove:
            os.remove(filename)

-----------------------------------------------------

-----------------------------------------------------
## B. SARSA data preparation: generate your maze based on the SOM data <a id='B'></a>
In Exercise 2, we obtained the SOM-lattice that basically plays the role of reducing the data. 
At this stage, we transition from the space of Cartesian coordinates of the robot environment to the SOM-lattice coordinates. This means that the robot will move straigth from one point of the SOM-lattice to another one. 
But not all of the points are available for transitioning. So, here we define which directions are available and we generate a reward matrix based on this information. You will go through the following steps:

- You can see 2 images wich represents the translation from Cartesian coordinates to SOM-lattice coordinates.
- Input the goal position that you want to use as [y,x] with respect to SOM-lattice coordinates.
- Analyse the Reward matrix you obtain. This matrix is of the form [N,N,4], where **N** is size of the SOM-lattice and **4** is number of possible actions (Down, Up, Right, Left). 

In [ ]:
from epflx_robox_nrp_utils.SARSA.SARSA_additional import SARSA_additional
sarsaad = SARSA_additional()
reward, actions = sarsaad.run_processing()

Run the next cell to save your result into your Collab storage.

In [ ]:
save_to_collab_storage(filepath='SOM_possible_actions.csv', filetype='text/csv', remove=False)

-----------------------------------------------------

-----------------------------------------------------
## C. SARSA data preparation: generate your maze based on the SOM data <a id='C'></a>

### C.1. SARSA description <a id='C1'></a>
****

**Exercise goal:**<br>
Your task is to implement a State-Action-Reward-State-Action algorithm (SARSA) to navigate the agent environment.

**Input:** SOM-lattice given by Exercise 2 (*'SOM_data_lattice.csv'*).<br>
In order to run the <font color=green>**_SARSA data preparation_**</font> cell, you have to generate a maze and a reward table ('SARSA_data_reward.csv') based on the SOM lattice data.<br>

**Output:** Expected reward matrix [Q-values] (*'SARSA_data_Qvalue.csv'*).<br>
When running the <font color=green>**_NRP platform test_**</font> cell, you can generate a random start position , or enter yours, and a list of way points (*'SARSA_data_way_points.csv'*) to reach a goal based on your SARSA learning.<br>
****

**SARSA algorithm:**<br>
*<font color=green>**while**</font> (N_trials are not done):*<br>
> <font color=blue>*# TRIAL: run_trial()*</font><br>
> *initialize eligibility*<br>
> *update policy (greedy epsilon)*<br>
> *generate random start position (take care the availability of that position)*<br>
> *make a choice of action: choose_action()*<br>
> *<font color=green>**while**</font> (agent is not in goal):*<br>
>> *<font color=blue># EPISODE: run_episode():</font>*<br>
>> *change current agent position: update_state()*<br>
>> *make a choice of action: choose_action()*<br>
>> *update the eligibility trace: update_E()*<br>
>> *update the expected reward: update_Q()*<br>
>> *update the latency parameter*<br>
>> *<font color=blue># ----------------------------------</font>*<br>
>> *checkup of trial end: agent in the goal (also, lattency limit can make trial end)*<br>

****


**Initialization functions:**<br>
 - **`init_parameters()`:** generate new order of robot positions; <font color=green>**return**</font> *__array__ of new order*.<br>
 - **`init_trial()`:** generate new order of robot positions; <font color=green>**return**</font> *__array__ of new order*.<br>
 
**Update functions:**<br>
 - **`update_state()`:** Remember the old position of the agent before moving on. Update the agents position according to the chosen action. Check if the agent has bumped into a wall (move to is_wall).; <font color=green>**return**</font> *__nothing__*.<br>
 - **`update_E()`:** Update the eligibility trace according to SARSA; <font color=green>**return**</font> *__nothing__*.<br>
 - **`update_Q()`:** Update the Q-values according to SARSA; <font color=green>**return**</font> *__nothing__*.<br>
 
**Interaction functions:**<br>
 - **`choose_action()`:** Choose the next action based on the current estimate of the Q-values. The parameter epsilon determines how often the agent chooses the action with the highest Q-value (probability `1 - epsilon`). In the rest of the cases, a random action is chosen; <font color=green>**return**</font> *__nothing__*.<br>
 - **`reward()`:** Evaluate which reward should be given when performing the chosen action at the current location; <font color=green>**return**</font> *__reward__ value for chosen action*.<br>

**Checkup functions:**<br>
 - **`is_trial_end()`:** checkup if it is the end trial; <font color=green>**return**</font> *boolean value*.<br>
 - **`is_bump()`:** checks if the agent bumps into a wall; <font color=green>**return**</font> *boolean value*.<br>
 - **`is_wall()`:** checks if the agent sets up on the wall; <font color=green>**return**</font> *boolean value*.<br>
****


**Global variables initialization (<font color=blue>default</font>, <font color=red>yours</font>):**<br>
 - <font color=blue>**self.video:**</font>visualization trigger (0 - no; 1 - mode1; 2 - mode2; 3 - latency).<br>
 - <font color=blue>**self.pos:**</font>data from input file ( <font color=red>*'NRP_data_robot_positions.csv'*</font> ).<br>
 - <font color=blue>**self.lattice:**</font> the initial randomly generated SOM.<br>
 - <font color=blue>**self.Nn:**</font>maze size, equal to SOM size [_Nn_ x _Nn_].<br>
 <br>
 - <font color=blue>**self.reward_position:**</font> the goal position. You can enter yours when running the <font color=green>**_SARSA data preparation_**</font> cell, otherwise the goal position will be generated randomly.<br>
 - <font color=blue>**self.Reward:**</font> the reward table taken from input file ( <font color=red>*'SARSA_data_reward.csv'*</font> ) that is generated by running of the <font color=green>**_SARSA data preparation_**</font> cell.<br>
 <br>
 - <font color=blue>**self.latency:**</font>latency of the current trial (initial value is 0).<br>
 - <font color=blue>**self.latency_list:**</font> is **empty list** by default. This list will accumulate latency of each trial.<br>
 - <font color=blue>**self.trial:**</font>index of the current trial (initial value is 0).<br>
<br>
 - <font color=blue>**[self.x_position, self.y_position]:**</font> current position of the agent (in terms of SARSA algorithm - **_S'_**).<br>
 - <font color=blue>**[self.x_position_old, self.y_position_old]:**</font> the previous position of the agent (in terms of SARSA algorithm - **_S_**).<br>
 - <font color=blue>**[self.x_start, self.y_start]:**</font>random position of the agent at the very begining of the trial.<br>
 - <font color=blue>**self.action:**</font>current chosen action of the agent (in terms of SARSA algorithm - **_A'_**).<br>
 - <font color=blue>**self.action_old:**</font>previous chosen action of the agent (in terms of SARSA algorithm - **_A_**).<br>
<br>
 - <font color=red>**self.N_trials:**</font>number of trials during learning.<br>
 - <font color=red>**self.Q:**</font>initial state of the expected reward matrix [*__Nn__* x *__Nn__* x __4__] (represents the 4 possible actions: **up, down, right, left**). <br>
 - <font color=red>**self.e:**</font> the initial state of the eligibility trace matrix [*__Nn__* x *__Nn__* x __4__].<br>
 <br>
 - <font color=red>**self.reward_at_target:**</font>reward given to the agent when reaching the goal.<br>
 - <font color=red>**self.reward_at_wall:**</font>reward (punishment) given to the agent when bumping into a wall.<br>
<br>
 - <font color=red>**self.epsilon:**</font>initial probability with which the agent chooses a random action. This makes sure the agent explores the maze (**Policy: epsilon greedy**).<br>
 - <font color=red>**self.tau:**</font>time constant of SARSA algorithm.<br>
 - <font color=red>**self.eta:**</font>learning rate of SARSA algorithm.<br>
 - <font color=red>**self.gamma:**</font>discount factor - quantifies how far into the future a reward is still considered important for the current action.<br>
 - <font color=red>**self.lambda_eligibility:**</font>decay factor for the eligibility trace; the default is 0., which corresponds to no eligibility trace at all.<br>
****

<font color=red>**TO DO:**</font><br>
**1)** Run the <font color=green>**_SARSA data preparation_**</font> cell. You can see the representations (mode1, mode2) of the maze generated based on your SOM-lattice solution. Please, enter the goal coordinates to get the reward table. Then, press enter to finish this stage.<br>
**2)** Fill the hole (<font color=red>**episode**</font>) in SARSA algorithm **`run_trial()`**.<br>
**3)** Fill <font color=red>Initialization functions</font> parameters <font color=red>(yours)</font> for the very begining learning **`init_parameters()`** and needed parameters for the start of each trial **`init_trial()`**.<br>
**4)** Implement the <font color=red>Update functions</font>: agent position, eligibility trace and expected reward.<br>
**5)** Implement the <font color=red>Interaction functions</font>: choose an action (baced on the policy) and get reward for this action.<br>
**6)** Implement the <font color=red>Checkup functions</font>: make the agent check itself [is_trial_end(); is_bump(); is_wall()]<br>
**7)** Run the <font color=green>**_NRP platform test_**</font> cell to generate a random start position (or enter yours) and way points (*'SARSA_data_way_points.csv'*) to reach the given goal based on your SARSA learning.<br>
**6)** Run <font color=green>**_NRP platform_**</font> cell, join the Neurorobotics platform and clone the template named *Exercise 3: Navigate the robot ...*. Select the *Experiment files* tab and upload the two csv files *'SOM_data_lattice.csv'*, *'SARSA_data_way_points.csv'* to your NRP Storage. Then, **run the experiment to see the result**. <br>
****

<font color=red>**IMPORTANT:**</font><br>
**1)** Please, keep in mind that the grading process has **a time limit of 60 seconds**.<br>
**2)** Once you are ready to submit, please, uncomment the first line of your script (`%%writefile SARSA_solution.py`) and run the cell that contains the script. This way your script will be saved as a file with the given name. **Then, submit this file.**

---------------------------------------

---------------------------------------

### C.2. SARSA function (TO DO) <a id='C2'></a>

In [ ]:
#%%writefile SARSA_solution.py 

# class SARSA (Sarsa_solution)                                                                                               
import numpy
import time
from pylab import *
import matplotlib.pyplot as plt


class SARSA:

	#########################################
	###        Main stages of SARSA       ###
	#########################################
    
	def __init__(self, video=0):
		# upload lattice information
		from epflx_robox_nrp_utils.SARSA.SARSA_additional import SARSA_additional
		self.sarsaad = SARSA_additional()

		self.video = video
		self.pos = self.sarsaad.upload_positions()
		self.lattice = self.sarsaad.upload_lattice()
		self.Nn = len(self.lattice[0])
        
		# initialize parameters
		self.init_parameters()
        
		# reward administered at a target location and when bumping into walls
		self.Actions, self.reward_position = self.sarsaad.upload_reward()#(self)Reward
        
		# list containing the times it took the agent to reach the target for all trials
		# serves to track the progress of learning
		self.latency_list = []

		# initialize the state and action variables
		self.x_position = None
		self.y_position = None
		self.action = None
		self.x_position_old = None
		self.y_position_old = None
		self.action_old = None
        

	def run_sarsa(self):
		T = time.time()
		for self.trial in range(self.N_trials):
			# run a trial and store the time it takes to the target
			self.latency = self.run_trial()
            
			# Add latency of last trial and visualize latency 
			if (self.trial % 1000 == 0 or self.trial == self.N_trials): self.sarsaad.save_Qvalue(self.Q)
			self.latency_list.append(self.latency)
			if self.video == 3: self.visualization()
                
		self.sarsaad.print_Qvalue(self.Q, self.reward_position,self.Actions)
		print 'Done. Simulation time is ', time.time() - T, '(s).'

        
	############################################
	###        Visualization of SARSA        ###
	############################################
    
	def visualization(self):
		# visualization of training
		if (0 < self.video < 3): 
			simdata = [self.Nn, self.trial, self.latency]
			actdata = [self.x_position, self.y_position, self.x_position_old, self.y_position_old]
			Rdata = self.reward_position
			Sdata = [self.x_start, self.y_start]
			Qdata = self.Q[self.x_position][self.y_position][:]
			self.sarsaad.visualization(self.video, simdata, actdata, Rdata, Sdata, Qdata)
		# visualization of latency
		if self.video == 3:
			if (self.trial % int(self.N_trials / 25) == 0 or self.trial == self.N_trials - 1):    
				self.sarsaad.latency(self.latency_list,self.N_trials,self.Nn)
				time.sleep(0.5)
        
        
	"""======================================================================================================="""
	"""                                                TO DO                                                  """
	"""======================================================================================================="""


	################################################
	"""             SARSA algorithm              """
	################################################
        
	def run_trial(self):
		# run the trial
		self.init_trial()
		while True:
			self.run_episode()
			# visualize learning process
			if (0 < self.video < 3): self.visualization()
			# check if agent position is a goal
			if self.is_trial_end(): break
		return self.latency
    
    
    
	################################################
	"""        Initialization  functions         """
	################################################
    
    
	def init_parameters(self): 
		# number of trials
		self.N_trials = 10000
		self.epsilon = 1.0
		self.tau = self.N_trials/4
		self.eta = 0.3
		self.gamma = 0.8
		self.lambda_eligibility = 0.6
        
		# initialize the Q-values and the eligibility trace
		self.Q = numpy.zeros((self.Nn,self.Nn,4))
		self.e = numpy.zeros((self.Nn,self.Nn,4))
        

	def init_trial(self):
		self.e = numpy.zeros((self.Nn,self.Nn,4))
		self.epsilon = np.exp(-self.trial/(self.tau/1.0))
        
		# choose the initial position
		while True:
			self.x_start = numpy.random.randint(0,self.Nn)
			self.y_start = numpy.random.randint(0,self.Nn)
			self.x_position = self.x_start; self.y_position = self.y_start
			if not self.is_block(): break

		self.choose_action()
		self.latency = 0.0

        
	def run_episode(self):
		self.update_state()
		self.choose_action()  
		self.update_E()
		self.update_Q()
		self.latency += 1

    
	#########################################
	"""        Updates functions      """
	#########################################
    
	def update_state(self):
		# remember the old position of the agent
		self.x_position_old = self.x_position
		self.y_position_old = self.y_position
        
		# update the agents position according to the action
		# move down
		if self.action == 0: self.x_position += 1
		# move up
		elif self.action == 1: self.x_position -= 1
		# move right
		elif self.action == 2: self.y_position += 1
		# move left
		elif self.action == 3: self.y_position -= 1
        
    
	def update_E(self):
		# update the eligibility trace
		self.e = self.lambda_eligibility * self.e
		self.e[self.x_position_old, self.y_position_old,self.action_old] += 1.
    
    
	def update_Q(self):
		# update the Q-values
		self.Q +=  self.eta * self.e * \
			          (self.reward() - \
			          (self.Q[self.x_position_old,self.y_position_old,self.action_old] - \
			           self.gamma * self.Q[self.x_position, self.y_position, self.action]))


	#############################################
	"""        Interaction functions        """
	#############################################
        
	def choose_action(self):
		# choose the next action
		self.action_old = self.action
		if numpy.random.rand() < self.epsilon:
			ind = numpy.random.randint(len(self.Actions[self.x_position][self.y_position]))
			self.action = self.Actions[self.x_position][self.y_position][ind]
		else:
			ind = argmax(self.Q[self.x_position,self.y_position,self.Actions[self.x_position][self.y_position]])
			self.action = self.Actions[self.x_position][self.y_position][ind]
    
    
	def reward(self):
		# Look at next positions, is it reward (goal)?
		# move down
		if self.action_old == 0: self.x_pos = self.x_position_old + 1; self.y_pos = self.y_position_old
		# move up
		elif self.action_old == 1: self.x_pos = self.x_position_old - 1; self.y_pos = self.y_position_old
		# move right
		elif self.action_old == 2: self.y_pos = self.y_position_old + 1; self.x_pos = self.x_position_old
		# move left
		elif self.action_old == 3: self.y_pos = self.y_position_old - 1; self.x_pos = self.x_position_old
            
		# return an actual reward
		if (self.x_pos == self.reward_position[0]) and (self.y_pos == self.reward_position[1]):
			return 1.0
		else: return 0.0

       
	#########################################
	"""       Checkup functions          """
	#########################################
        
	def is_trial_end(self):
		return (self.reward() == 1.0) or (self.latency >= self.Nn * self.Nn)


	def is_block(self):
		# if action is impossible
		if len(self.Actions[self.x_position][self.y_position]) == 0.0:
			return True
		else: return False

****
### C.3. Upload SARSA function into the storage <a id='C3'></a>
Once you are done and ready to submit your solution, please, follow the next steps: </br>

- Move to [C.2. SARSA function](#C2) and uncomment the first line, i.e. remove the `#` symbol before `%%writefile SARSA_solution.py` ;
- Run cell [C.2. SARSA function](#C2) then you sholud see `Writing SARSA_solution.py` just above; 
- Run cell [C.3. Upload SARSA function into the storage](#C3) to save this file in your Collab storage (check your Collab storage to see if the file is there).

In [ ]:
save_to_collab_storage(filepath='SARSA_solution.py', filetype='text/x-python', remove=True)

---------------------------

---------------------------
## D. SARSA simulation <a id='D'></a>
In this part, you can test your implementation of **C.2. SARSA function (TO DO)** using different modes of visualization. You can also save the results of Q-value calculations into a file and upload this file to your Collab storage.

### D.1. Perform SARSA training (editable) <a id='D1'></a>
The cell below run [your script](#C2) so as to simulate SARSA training. You can visualize this process.
- *sarsa = SARSA(visualization)* # upload your SARSA function [**visualization**: No - **0**, Mode-1 - **1**, Mode-2 - **2**, Latency - **3**];</br> 
- *sarsa.run_sarsa()* # run simulation.

In [ ]:
sarsa = SARSA(3)
sarsa.run_sarsa()

### D.2. Save SARSA training result <a id='D2'></a>
The result of the SARSA training is a *navigation heatmap* based on Q-values. This navigation heatmap is represented by a table after the training simulation. The Q-values have been saved into the file __*'SARSA_data_Qvalue.csv'*__ which is now contained in this Jupyter notebook. The next command will upload the __*'SARSA_data_Qvalue.csv'*__ to your Collab storage. 
Run the cell below and check your Collab storage to see if the __*'SARSA_data_Qvalue.csv'*__ is there.

In [ ]:
save_to_collab_storage(filepath='SARSA_data_Qvalue.csv', filetype='text/csv', remove=False)

-------------------------------

-------------------------------
## E. SARSA Evaluation <a id='E'></a>

The goal of the SARSA training is to obtain a navigation map based on Q-values. Such a map should provide the shortest path between any position of the robot environment and any goal position. The following command collects information about robot movements using this *Q-navigation map*. By running this comand, you will get the statistics of all robot paths to the goal. You can visualize this process and check how the robot performs.

In [ ]:
from epflx_robox_nrp_utils.SARSA.SARSA_evaluation import SARSA_evaluation
sarsaev = SARSA_evaluation()
# Run evaluation with a specific mode
from IPython.display import clear_output
try: video = input("To visualize the evaluation process, please, input '1': ")
except: video = 0;
clear_output()

# Analysis
best, good, over, never = sarsaev.run_evaluation(video)
# Statistics
print 'Agent came to the goal in', best+good, 'of', best+good+never,'cases.'
print 'Agent came to the goal by the shortest way in', best,'cases.'
print 'Agent came to the goal by non-shortest way in', good,'cases using', over, 'steps over.'
print 'Agent did not come to the goal in', never, 'cases.'

-----------------------------------------------

-----------------------------------------------
## F. Robot navigation within NRP platform (SOM & SARSA application) <a id='F'></a>
The goal of this last section is to reuse the SARSA training results in a robotics simulation.

### F.1. Generate test for NRP platform <a id='F1'></a>

In [ ]:
from epflx_robox_nrp_utils.SARSA.SARSA_evaluation import SARSA_evaluation
sarsaev = SARSA_evaluation()
sarsaev.test_generation()

### F.2. Save NRP test <a id='F2'></a>
We have just generated a path for the robot navigation within the NRP platform. 
This path is written into the file __*'SARSA_data_way_points.csv'*__ which is now contained in this Jupyter notebook space and consists of [x,y] coordinates of the positions the robot will go through. The following command upload the __*'SARSA_data_way_points.csv'*__ into your Collab storage. 

Run the cell below and check your Collab storage to see if the __*'SARSA_data_way_points.csv'*__ is there.

In [ ]:
save_to_collab_storage(filepath='SARSA_data_way_points.csv', filetype='text/csv', remove=False)

### F.3. Run experiment within NRP platform <a id='F3'></a>

Here we use the results obtained with the SOM and SARSA algorithms. Thanks to these results, we will navigate a simulated robot within a virtual environment of the [Neurorobotics platform](http://148.187.97.48/#/esv-private?dev). To launch the relevant simulation, please, follow these instructions below.

- Download the files __'SOM_lattice_data.csv'__ and __'SARSA_data_way_points.csv'__ from your Collab storage.
- Join the [Neurorobotics platform](http://148.187.97.48/#/esv-private?dev) and select the "Templates" tab.
- Select *Exercise_3* in the list of Templates (use the search filter) and press the *Clone* button.
- Select the "Experiment files" tab and open the *Exercise_3* folder.
- Upload the files __'SOM_lattice_data.csv'__ and __'SARSA_data_way_points.csv'__ into the Experiment files storage.
- Move back to the "My experiments" tab and launch *Exercise_3*.
- Run the simulation and check if the robot navigates through the specfified waypoints.

## G. Submission and grading
Run the next cell to submit your SARSA implementation. Each submission is automatically tested and evaluated. Only the last submission will be taken into account for your grade.

In [ ]:
clients = get_hbp_service_client()
collab_path = get_collab_storage_path()
print("Retrieving your HBP OIDC token ...")
token = str(oauth.get_token())
print("token retrieved!")
submission_info = {
    'subheader': 'Exercise 3',
    'filepath': 'SARSA_function.py',
    'token': token,
    'collab_path': collab_path,
    'clients_storage': clients.storage
}
from epflx_robox_nrp_utils.submission_manager.submission_widget import display_submission_widget
display_submission_widget(submission_info)